In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

In [4]:

def read_csv(file_path):
    '''
    Read CSV file and return data as numpy array
    '''
    df = pd.read_csv(file_path)
    if "class" in list(df.columns.values):
        df = df.drop("class", axis=1)
    data = np.array(df,dtype=object)
    data = data.astype(float)
    return data

In [5]:
data = read_csv("init_demo.csv")
data.shape

(16, 300)

In [6]:
def find_max_for_one_file(data, var):
    current_max = 0
    vars = {"x": 0, "y": 1, "z": 2}
    for i in range(data.shape[0]):
        frame = data[i]
        frame = frame.reshape((-1, 4))
        for j in range(frame.shape[0]):
            if frame[j][vars[var]] > current_max:
                current_max = frame[j][vars[var]]
    
    return current_max
find_max_for_one_file(data, "x")

0.8602175712585449

In [7]:
def initialize_matrix_for_frame(data, idx, x_dim, y_dim):
   
    # create a matrix of all zeros
    matrix = np.zeros((x_dim,y_dim))
    # fill the matrix with random values
    frame = data[idx]
    frame = frame.reshape((-1, 4))
    for j in range(len(frame)):
        matrix[int(frame[j][0] * 1000)][int(frame[j][1] * 1000)] = j
    return matrix

In [132]:
matrix = initialize_matrix_for_frame(data, 0)

TypeError: initialize_matrix_for_frame() missing 2 required positional arguments: 'x_dim' and 'y_dim'

In [8]:
# plot the matrix
coords = np.nonzero(matrix)

# Plot the non-zero elements
plt.scatter(coords[0], coords[1])

# Set the x and y limits
plt.xlim(0, matrix.shape[0])
plt.ylim(0, matrix.shape[1])

# Set the labels for the axes
plt.xlabel('Row number')
plt.ylabel('Column number')

# Show the plot
plt.show()

NameError: name 'matrix' is not defined

In [8]:
def generate_tensor(data, x_dim=1500, y_dim=3200, sliding_window_size=10):
    # check for max value in x and y, which will then be used for the shape of the matrix

    for i in range(len(data) - sliding_window_size + 1):
        # print(i, len(data) - sliding_window_size + 1)
        tensor = np.zeros((sliding_window_size, x_dim,y_dim))
        for j in range(i, i + 10):
            new_matrix = initialize_matrix_for_frame(data, j, x_dim, y_dim)
            tensor[j-i] = new_matrix
        yield tensor
    

In [18]:
test_arr = np.empty((0,10,1500,3200))
tensor_gen = generate_tensor(data, 1500, 3200, sliding_window_size=10)
arr_to_add = np.expand_dims(next(tensor_gen), axis=0)
test_arr = np.concatenate((test_arr,  arr_to_add), axis=0)
test_arr.shape

(1, 10, 1500, 3200)

In [22]:

path = "/Users/aly/Documents/Programming/Apps/Machine Learning/ASL Converter/data_augmentation/data_four_labels_augmentation"

def find_max_dim__for_entire_folder(path, sliding_window_size = 10):
    # The first thing to do is find the maximum x and y values for our matrices
    max_x_dim = 0
    max_y_dim = 0
    for label in os.listdir(path):
        if label != ".DS_Store":
            i = 1
            j = 0
            for file in os.listdir(os.path.join(path, label)):
                # pick 25% of the data
                if i%4 == 0:
                    data = read_csv(os.path.join(path, label, file))
                    max_x = find_max_for_one_file(data, "x") * 1000
                    max_y = find_max_for_one_file(data, "y") * 1000
                    x_dim = round(math.ceil(max_x/100)*100, -1)
                    y_dim = round(math.ceil(max_y/100)*100, -1)
                    if max_x_dim < x_dim:
                        max_x_dim = x_dim
                    if max_y_dim < y_dim:
                        max_y_dim = y_dim
                i += 1
        print(label, max_x_dim, max_y_dim)
    return max_x_dim, max_y_dim
# find_max_dim__for_entire_folder(path, sliding_window_size = 10)
                            

In [25]:
 # 80/20 split
x_dim = 1500
y_dim = 3200
sliding_window_size = 10

def process_entire_folder(path, sliding_window_size = 10, x_dim=1500, y_dim=3200):
    # The first thing to do is find the maximum x and y values for our matrices
    global X_train
    global y_train
    global X_val
    global y_val
    X_train = np.empty((0,10,x_dim,y_dim))
    y_train = []
    X_val = np.empty((0,10,x_dim,y_dim))
    y_val = []
    for label in os.listdir(path):
        if label != ".DS_Store":
            i = 1
            j = 0
            for file in os.listdir(os.path.join(path, label)):
                # pick 25% of the data
                if i%4000 == 0:
                    j += 1
                    print(j, i, label, file)
                    tensor_generator = generate_tensor(data, x_dim, y_dim, sliding_window_size)
                    for tensor in tensor_generator:
                        arr_to_add = np.expand_dims(tensor, axis=0)
                        if j%5 == 0:
                            # add to validation set
                            X_val = np.concatenate((X_val,  arr_to_add), axis=0)
                            y_val.append(label)
                        else:
                            X_train = np.concatenate((X_train,  arr_to_add), axis=0)
                            y_train.append(label)
                i += 1
process_entire_folder(path, sliding_window_size, x_dim, y_dim)

1 400 dog FBD4NFz4QaA4056_reflection_rotation_15_translate_0.5_left.csv_projective_geo_9_True.csv
2 800 dog Ag5VPvAe8PA5557_translation_0.5_scale_0.2_translate_0.5_left.csv
3 1200 dog N5ZUQxL4__c0_reflection_rotation_15_translation_0.5.csv
4 1600 dog N5ZUQxL4__c0_reflection_translation_0.5_translate_0.5_left.csv_projective_geo_14_False.csv
5 2000 dog rIlQQTN2b586887_reflection_rotation_15_translation_0.5_scale_0.2.csv
6 2400 dog 498gWK7nFCI20_reflection_rotation_15_translation_0.5_scale_0.2.csv_projective_geo_12_False.csv
7 2800 dog 1E8k8gI_xYk3420_rotation_15_translation_0.5_scale_0.2_translate_0.5_left.csv_projective_geo_13_False.csv
8 3200 dog rIlQQTN2b586887_rotation_15_translation_0.5_translate_0.5_left.csv
1 400 milk rkQZQhloXuE4918_reflection_rotation_15_scale_0.2.csv_projective_geo_13_False.csv
2 800 milk 2sGQuduhAf43239_rotation_15_translation_0.5_scale_0.2_translate_0.5_left.csv_projective_geo_14_False.csv


KeyboardInterrupt: 

: 

: 

In [21]:
X_train

[]